In [ ]:
! pip install spacy

In [1]:
import pandas as pd

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [4]:
df = pd.read_json('/content/drive/MyDrive/all.jsonl', lines=True)

In [7]:
del df['id']

In [8]:
df

,text,cats,entities,Comments
0,Название дисциплины: Комбинаторика и теория гр...,"[Fullstack-разработчик, ML-разработчик]","[[134, 162, Method], [253, 279, Method], [377,...",[]
1,Название дисциплины: Коммерциализация результа...,[Аналитик],"[[84, 147, Technology], [166, 184, Method], [4...",[]
2,Название дисциплины: Учебная практика (проектн...,[],"[[9, 19, Tool]]",[]
3,Название дисциплины: Производственная практика...,[Инженер-данных],"[[72, 94, Tool], [254, 276, Technology], [279,...",[]
4,Название дисциплины: Коммерциализация результа...,[Менеджер продукта],"[[136, 146, Technology], [174, 184, Method], [...",[]
...,...,...,...,...
63,Название дисциплины: Основы научных исследован...,[],"[[714, 722, Tool], [780, 858, Method], [861, 9...",[]
64,Название дисциплины: Программные средства обра...,"[ML-разработчик, Разработчик САУ]","[[1060, 1112, Method], [1135, 1181, Method], [...",[]
65,Название дисциплины: Математическое моделирова...,[],"[[205, 243, Technology], [245, 265, Technology...",[]
66,Название дисциплины: Цифровая обработка данных...,[],"[[147, 191, Tool], [192, 220, Tool], [221, 227...",[]


In [20]:
data = df[['text','entities']].to_dict()

In [19]:
print(data['text'][0])
print(data['entities'][0])

Название дисциплины: Комбинаторика и теория графов
Много комбинаторных вопросов исторически рассматривались изолированно, представляя специальное решение проблемы, возникшей в некотором математическом контексте. В конце двадцатого века были разработаны общие теоретические методы, превратившие комбинаторику в независимую отрасль математики. Дисциплина объединяет классические комбинаторные идеи и их обобщения с прикладной проблематикой, в том числе, генерированием комбинаторных объектов, кодированием. Обсуждается техника работы с производящими функциями. Большой раздел связан с базовыми понятиями теории графов и примерами алгоритмов на графах. Этот раздел можно назвать «прикладной теорией алгоритмов», так как в нем на важных примерах обсуждаются общие принципы доказательства корректности алгоритмов и их эффективности. . Формирование достаточно высокой математической культуры является основной целью математической подготовки. Для этого необходимо, познакомить студентов с основными понятия

In [37]:
training_data = {'classes' : ['Method', 'Technology', 'Tool', 'SoftSkills'], 'annotations' : []}

for iter in range(len(data)):
  temp_dict = {}
  temp_dict['text'] = data['text'][iter]
  temp_dict['entities'] = data['entities'][iter]
  training_data['annotations'].append(temp_dict)


print(training_data['annotations'][0])

{'text': 'Название дисциплины: Комбинаторика и теория графов\nМного комбинаторных вопросов исторически рассматривались изолированно, представляя специальное решение проблемы, возникшей в некотором математическом контексте. В конце двадцатого века были разработаны общие теоретические методы, превратившие комбинаторику в независимую отрасль математики. Дисциплина объединяет классические комбинаторные идеи и их обобщения с прикладной проблематикой, в том числе, генерированием комбинаторных объектов, кодированием. Обсуждается техника работы с производящими функциями. Большой раздел связан с базовыми понятиями теории графов и примерами алгоритмов на графах. Этот раздел можно назвать «прикладной теорией алгоритмов», так как в нем на важных примерах обсуждаются общие принципы доказательства корректности алгоритмов и их эффективности. . Формирование достаточно высокой математической культуры является основной целью математической подготовки. Для этого необходимо, познакомить студентов с основн

In [38]:
training_data.keys()

dict_keys(['classes', 'annotations'])

In [41]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("ru") # load a new spacy model
doc_bin = DocBin()

In [45]:
from spacy.util import filter_spans

for training_example  in tqdm(training_data['annotations']):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
      span = doc.char_span(start, end, label=label, alignment_mode="contract")
      if span is None:
          print("Skipping entity")
      else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)

doc_bin.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 2/2 [00:00<00:00, 30.29it/s]


In [47]:
! python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [48]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    957.78    0.00    0.00    0.00    0.00
 25     200        931.96  23663.87   98.95   98.44   99.47    0.99
 50     400        201.21    471.07   99.21   99.47   98.95    0.99
 75     600        101.10    374.41   99.47   99.47   99.47    0.99
100     800        122.48    262.25   99.47   99.47   99.47    0.99
125    1000        122.33    227.23   99.47  100.00   98.95    0.99
150    1200        107.24    229.64   99.47  100.00   98.95    0.99
175    1400         20.96    179.89   99.47   99.47   99.47    0.99
200    1600         55.06    196.13 

In [51]:
nlp_ner = spacy.load("model-best")
doc = nlp_ner(training_data['annotations'][0].get('text'))
spacy.displacy.render(doc, style="ent", jupyter=True)